In [1]:
import torch
import torch.nn as nn
import annmodel
import snnmodel

def estimate_power(model, input_tensor, T=8):
    # 假设每次乘法和加法消耗的能量 (单位: Joules)
    power_per_multiply = 1e-9  # 假设每次乘法操作消耗的能量
    power_per_add = 1e-9  # 假设每次加法操作消耗的能量

    total_multiplications = 0
    total_additions = 0

    def count_ops(layer, x):
        nonlocal total_multiplications, total_additions
        if isinstance(layer, nn.Conv2d):
            # 计算卷积操作的乘法次数和加法次数
            out_size = x.shape[2:]  # 输出大小
            in_channels = layer.in_channels
            out_channels = layer.out_channels
            kernel_size = layer.kernel_size[0]
            total_multiplications += out_size[0] * out_size[1] * in_channels * kernel_size * kernel_size * out_channels
            total_additions += total_multiplications  # 每次乘法后都会有加法
        elif isinstance(layer, nn.Linear):
            # 计算全连接层的乘法次数和加法次数
            total_multiplications += x.shape[1] * layer.out_features
            total_additions += total_multiplications  # 每次乘法后都会有加法
        return x

    def forward_hook(module, input, output):
        # 钩子函数，用于计数每一层的操作
        count_ops(module, input[0])

    # 给每一层添加钩子函数
    hooks = []
    for layer in model.modules():
        hook = layer.register_forward_hook(forward_hook)
        hooks.append(hook)

    # 前向传播
    model(input_tensor)

    # 删除钩子
    for hook in hooks:
        hook.remove()

    # 假设每个乘法和加法操作的能量消耗
    total_power = (total_multiplications + total_additions) * (power_per_multiply + power_per_add)
    return total_power

# 假设一个输入张量
T = 4
input_tensor = torch.randn(64, T, 3, 32, 32)
snn = snnmodel.DPN(segment=2, T=T)
ann = annmodel.DPN(segment=2)

# 估算功耗
power_consumption = estimate_power(snn, input_tensor)
print(f"SNN's estimated power consumption for one forward pass: {power_consumption:.9f} Joules")

power_consumption = estimate_power(ann, input_tensor)
print(f"ANN's estimated power consumption for one forward pass: {power_consumption:.9f} Joules")


/home/TuYuqing/anaconda3/envs/snn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SNN's estimated power consumption for one forward pass: 0.107464704 Joules
ANN's estimated power consumption for one forward pass: 0.211943424 Joules
